# A look into Lucene Scoring functions

Apache Lucene is a Java library used for the full text search of documents, and is at the core of search servers such as [Solr](http://lucene.apache.org/solr/) and [Elasticsearch](https://www.elastic.co/products/elasticsearch).

In a nutshell, Lucene allows you to run a query against a set of documents and receive back a ranked result of the documents, each with a score calculated based on how similar the document matched the original query.

In this Notebook I have rewritten in Python the two most important similarities as they are implemented by Lucene:
* The **BM25**, which has become the default similarity from Lucene 6 (released in April 2016)
* The **TF-IDF**, the classical similarity used in information retriavial which was the default similarity in Lucene up to version 6

## 1. Classic Lucene Similarity: a modified TF-IDF

In information retrieval, [**tf–idf**](https://en.wikipedia.org/wiki/Tf%E2%80%93idf), short for **term frequency–inverse document frequency**, is a numerical statistic that is intended to reflect how important a word is to a document in a collection or corpus.

Lucene's Practical Scoring Function uses a modified version of the TF-IDF found in textbooks, which is explained below. 

### Lucene modified TF

Instead of taking the Term Frequency as is, Lucene uses $\sqrt{TF}$. The TF score is computed as follows:

| TF | Lucene TF |
|------|------|
|   1  | 1 |
|   2  | 1.414 |
|   4  | 2 |
|   8  | 2.828 |
|   16  | 4 |

This way, documents with twice the number of terms as another document aren’t twice as relevant.

### Lucene modified IDF

Similarly users don’t consider terms that only occur in 10 documents ten times as special as those that occur in 100 documents. Instead, the IDF score is computed as:

$$ln\left ( \frac{docCount}{docFreq_{t} + 1} \right ) + 1$$

where *docCount* is the total number of documents in the collection, and *docFreq<sub>t</sub>* is the number of documents in the collection containing  term *t*.

### Document length

The impact of a document’s length is taken into account by multiplying the TF\*IDF score by $\frac{1}{\sqrt{|d|}}$


### Lucene Practical Scoring Function

Combinining the pieces together and normalising, Lucene's Practical Scoring Function (simplified version) is as follows: 

$$score_{q,\; d}   =   coord_{q, d} \;  \times \;  queryNorm_{q} \;  \times \; \sum_{t\;  in\; q} \left ( tf_{t, d}\;  \times \; idf_{t}^{2} \;  \times \;  norm_{d}  \right )$$

where:

* **coord<sub>q, d</sub>** : is a score factor based on how many of the query terms are found in the specified document. For example, if documnt *d* contains 2 of the 3 terms of query *q*, then coord<sub>q, d</sub> is 2/3.
* **queryNorm<sub>q</sub>** : is a normalizing factor used to make scores between queries comparable. This factor does not affect document ranking (since all ranked documents are multiplied by the same factor), but rather just attempts to make scores from different queries (or even different indexes) comparable. The default computation produces a Euclidean norm:

$$ queryNorm_{q} =\frac{1}{\sqrt{sumOfSquaredWeights}}   $$

    where

$$ sumOfSquaredWeights = \sum_{t \; in \; q} idf_{t}^{2} $$

* **tf<sub>t, d</sub>**: correlates to the term's frequency, defined as the number of times term t appears in the currently scored document d. Documents that have more occurrences of a given term receive a higher score. It is computed as explained above.

* **idf<sub>t</sub>** : stands for Inverse Document Frequency. This value correlates to the inverse of docFreq (the number of documents in which the term t appears). This means rarer terms give higher contribution to the total score. idf<sub>t</sub> appears for t in both the query and the document, hence it is squared in the equation. It is computed as explained above.

* **norm<sub>d</sub>** : encapsulates a (indexing time) length factor computed and stored as an 8 bit floating point value which causes some loss of precison (lossy compression).



## 2. New Lucene Default Similarity: BM25

BM25 stands for [**Best Matching 25**](https://en.wikipedia.org/wiki/Okapi_BM25), also called *Okapi weighting scheme*, and improves upon the classical TF \* IDF.

Released in [1994](http://trec.nist.gov/pubs/trec3/t3_proceedings.html), BM25 has its roots in [probabilistic information retrieval](http://nlp.stanford.edu/IR-book/html/htmledition/probabilistic-information-retrieval-1.html). A relevance score, according to probabilistic information retrieval, reflects the probability a user will consider the result relevant. 

The [BM25 weighting scheme](https://github.com/apache/lucene-solr/blob/master/lucene/core/src/java/org/apache/lucene/search/similarities/BM25Similarity.java) has become the default similarity measure from **Lucene 6**.

### Lucene BM25 view of IDF

The [original BM25 formula for IDF](https://en.wikipedia.org/wiki/Okapi_BM25#The_ranking_function) is computed as:

$$idf_{t} = \ln (\frac{docCount - docFreq_{t} + 0.5}{docFreq_{t} + 0.5})$$

where *docCount* is the total number of documents in the collection, and *docFreq<sub>t</sub>* is the number of documents in the collection containing  term *t*.

It shows potentially major drawbacks when using it for terms appearing in more than half of the corpus documents. These terms' IDF is negative, so for any two almost-identical documents, one which contains the term and one which does not contain it, the latter will possibly get a larger score. This means that terms appearing in more than half of the corpus will provide negative contributions to the final document score. This is often an undesirable behavior, so Lucene overcomes this problem by adding 1 to the value, before taking the log, which makes it impossible to compute a negative value.

$$idf_{t} = \ln (1 + \frac{docCount - docFreq_{t} + 0.5}{docFreq_{t} + 0.5})$$




### Lucene BM25 view of TF and the influence of docLength

Term frequency in BM25 lower the impact of term frequency even further than traditional TF \* IDF. In the BM25, the impact of term frequency is always increasing, but asymptotically approaches the value (k + 1) (where in the default implementation k = 1.2).

$$ \frac{(k + 1) \times tf_{t, d}}{k + tf_{t, d}} $$

More *tf* always means more relevance. However in this way it quickly hits diminishing returns. Classic *tf*, on the other hand, constantly increases and never reaches a saturation point.

Changing k can be a useful tuning approach to modify the impact of the *tf*. A higher k causes *tf* to take longer to reach saturation. By stretching out the point of saturation, it stretches out the relevance difference between higher and lower term frequency docs.

The TF score above is further influenced by whether the document length is above or below the average length of a document in the corpus.
The formula above is modified by adding (1.0 - b + b * L) as a multiple on k in the denominator.

$$ tfNorm_{t,\; d} =  \frac{(k + 1) \times  tf_{t, d}}{k \times  (1.0 - b + b \times  L) + tf_{t, d}} $$

Here L is how long a document is relative to the average document length. L therefore is actually presented as $ \frac{|d|}{avgDocumentLength} $, i.e. this document length divided by the average document length.

The constant b (where in the default implementation b = 0.75) allows to finely tune how much influence the L value has on scoring. A b of 0 completely removes the influence of L. A higher b adds more document length influence on the scoring. 

### All Together

Putting all together, the score of a document *d* given a query *q* (with multiple terms) is given by:

$$ score_{q,\; d} = \sum_{t\; in \; q }tfNorm_{t,\; d} \times  idf_{t} $$


# A note on Lucene Lossy Compression

Lucene computes the lenght of a document |d| at indexing time, i.e. every time a document is added to the index. This information is then retrieved at search time, i.e. when the query is performed. 

However, to efficiently store this information, Lucene performs a lossy compression. The document length value is encoded as a single byte before being stored. At search time, the byte value is read from the index directory and decoded back to a float value. This encoding/decoding, while reducing index size, comes with the price of precision loss, i.e. it is not guaranteed that $decode( encode(x) ) = x$.

The rationale supporting such lossy compression is that given the difficulty (and inaccuracy) of users to express their true information need by a query, only big differences matter.

# Implementation

In [3]:
import os
import tqdm
import struct
import numpy as np
import pandas as pd
from scipy import stats
from scipy import sparse

from sklearn import metrics
from sklearn.cross_validation import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

In [4]:
# Implement encoding and decoding function for the lossy compression, as done here 
# https://github.com/apache/lucene-solr/blob/master/lucene/core/src/java/org/apache/lucene/util/SmallFloat.java
def floatToRawIntBits(f):
    s = struct.pack('=f', f)
    return struct.unpack('=l', s)[0]

def intBitsToFloat(b):
    s = struct.pack('>l', b)
    return struct.unpack('>f', s)[0]

def byte315ToFloat(b):
    if (b == 0):
        return 0.0
    bits = (b&0xff) << (24-3)
    bits += (63-15) << 24
    return intBitsToFloat(bits)

def floatToByte315(f):
    bits = floatToRawIntBits(f)
    
    smallfloat = bits >> (24-3)
    
    if (smallfloat <= ((63-15)<<3)):
        return  bytes(0) if (bits<=0) else bytes(1)
    
    if (smallfloat >= ((63-15)<<3) + 0x100):
        return -1
    
    return int(bytes(smallfloat - ((63-15)<<3)))

In [5]:
class Similarity(object):
    def __init__(self): 
        self.NORM_TABLE = np.arange(256, dtype= float)
    
    def decodeNormValue(self, b):
        return self.NORM_TABLE[b & 0xFF] # & 0xFF maps negative bytes to positive above 127
    
    def encodeNormValue(self, value):
        pass
    
    def get_idf(self, docCount, docFreqs):
        pass
    
    def get_tf(self, docs, query):
        vect = CountVectorizer(vocabulary = query, analyzer='word')
        tf = vect.fit_transform(docs)
        docCount = tf.shape[0]
        docFreqs = (tf != 0).sum(0)
        return (tf, docCount, docFreqs)
    
    def execute(self, query, docs):
        pass
    
    def transform(self, value):
        pass
    
    def get_coord(self, tf, idf):
        nTerms = tf.shape[1]
        return np.divide((tf != 0).astype(float)*(idf != 0).astype(float), nTerms)
    
    def get_norm(self, docs):
        vect = CountVectorizer(analyzer='word')
        X = vect.fit_transform(docs)
        docCount = X.shape[0]

        avgFieldLength = X.sum()/float(docCount)
        
        norm = np.matrix(map(self.decodeNormValue, 
                             map(self.encodeNormValue, 
                                 map(self.transform, X.sum(axis = 1))))).reshape(docCount, 1)
        
        return (norm, avgFieldLength)
    
    def score(self, query, docs):
        query = query.split(' ')
        scores = self.execute(query, docs)
        return scores

In [6]:
class ClassicSimilarity(Similarity):
    def __init__(self):
        Similarity.__init__(self)
    
        for i in range(256):
            self.NORM_TABLE[i] = byte315ToFloat(int(bytes(i)))

    def encodeNormValue(self, value):
        return floatToByte315(value)
    
    def get_idf(self, docCount, docFreqs):
        idf = 1.0 + np.log(np.divide(docCount, (docFreqs + 1.0)))
        return np.square(idf, idf).T
    
    def get_coord(self, tf, idf):
        nTerms = tf.shape[1]
        return np.divide((tf != 0).astype(float)*(idf != 0).astype(float), nTerms)
    
    def transform(self, value):
        return 1.0/np.sqrt(value)
        
    def execute(self, query, docs):
        tf, docCount, docFreqs = self.get_tf(docs, query)
        idf = self.get_idf(docCount, docFreqs)
        tf = tf.sqrt()
        coord = self.get_coord(tf, idf)
        norm, avgFieldLength = self.get_norm(docs)
        queryNorm = np.divide(1.0, np.sqrt(idf.sum(axis = 0)))
        # coord * dot(tf * norm, idf * queryNorm)
        tf = tf.multiply(norm)
        idf = np.multiply(idf, queryNorm)
        tfidf = tf.dot(idf)
        tfidf = np.multiply(tfidf, coord)
        return tfidf
        

In [7]:
class BM25(Similarity):
    def __init__(self, k = 1.2, b = 0.75, coord_factor = False):
        Similarity.__init__(self)
        self.k = k
        self.b = b
        self.coord_factor = coord_factor # multiply the scores by the coord factor: 
                                        # n° query terms in the document / total n° of terms in the query
    
        for i in range(1, 256):
            f = byte315ToFloat(int(bytes(i)))
            self.NORM_TABLE[i] = 1.0 / (f*f)
        self.NORM_TABLE[0] = 1.0 / self.NORM_TABLE[255]

    def encodeNormValue(self, value):
        boost = 1.0
        return floatToByte315(boost / float(np.sqrt(value)))
    
    def get_idf(self, docCount, docFreqs):
        idf = np.log(1 + np.divide((docCount - docFreqs + 0.5), (docFreqs + 0.5)))
        return idf.T
    
    def transform(self, value):
        return value
        
    def execute(self, query, docs):
        tf, docCount, docFreqs = self.get_tf(docs, query)
        fieldLengths, avgFieldLength = self.get_norm(docs)
        tfNorm = np.divide((tf * (self.k + 1)).toarray(),
                           (tf + self.k * (1 - self.b + self.b * (fieldLengths/avgFieldLength))))
        idf = self.get_idf(docCount, docFreqs)
        if self.coord_factor == True:
            coord = self.get_coord(tf, idf)
        else:
            coord = 1.0
        # dot(tf, idf)
        tfidf = tfNorm.dot(idf)
        return np.multiply(tfidf, coord)
    

In [8]:
def print_rank(docs, scores):
    candidates = scores != 0
    indices = docs.index[np.asarray(candidates.T)[0,:]]
    sorted_indices = indices[np.argsort(-np.asarray(scores[indices].T)[0,:])]
    for rank, idx in enumerate(sorted_indices):
        print "%d.\t%s\t%f" %(rank, docs[idx], scores[idx, 0])

# Example

In [9]:
# Some dummies documents
docs = pd.Series(["Lucene Action continue continued", "Lucene  Dummies mbuy", "Managing Gigabytes", 
                  "Art  Computer Science", "Action", "Lucene way", "Managing Megabytes lucene", "Art Gaming"])

### BM25

In [10]:
# With no coord factor
bm25 = BM25()
scores = bm25.score('lucene action', docs)
print_rank(docs, scores)

0.	Action	1.697623
1.	Lucene Action continue continued	1.585029
2.	Lucene way	0.686408
3.	Lucene  Dummies mbuy	0.556542
4.	Managing Megabytes lucene	0.556542


In [11]:
scores = bm25.score('lucene', docs)
print_rank(docs, scores)

0.	Lucene way	0.686408
1.	Lucene Action continue continued	0.556542
2.	Lucene  Dummies mbuy	0.556542
3.	Managing Megabytes lucene	0.556542


In [12]:
scores = bm25.score('lucene way', docs)
print_rank(docs, scores)

0.	Lucene way	2.460747
1.	Lucene Action continue continued	0.556542
2.	Lucene  Dummies mbuy	0.556542
3.	Managing Megabytes lucene	0.556542


In [13]:
# With coord factor. See w.r.t Out[10]
bm25c = BM25(coord_factor = True)
scores = bm25c.score('lucene action', docs)
print_rank(docs, scores)

0.	Lucene Action continue continued	1.585029
1.	Action	0.848812
2.	Lucene way	0.343204
3.	Lucene  Dummies mbuy	0.278271
4.	Managing Megabytes lucene	0.278271


### Modified TF-IDF

In [14]:
tfidf = ClassicSimilarity()

In [15]:
scores = tfidf.score('lucene action', docs)
print_rank(docs, scores)

0.	Lucene Action continue continued	1.233349
1.	Action	0.795332
2.	Lucene way	0.273761
3.	Lucene  Dummies mbuy	0.219009
4.	Managing Megabytes lucene	0.219009


In [16]:
scores = tfidf.score('lucene', docs)
print_rank(docs, scores)

0.	Lucene way	0.918752
1.	Lucene Action continue continued	0.735002
2.	Lucene  Dummies mbuy	0.735002
3.	Managing Megabytes lucene	0.735002


In [17]:
scores = tfidf.score('lucene way', docs)
print_rank(docs, scores)

0.	Lucene way	1.751708
1.	Lucene Action continue continued	0.192750
2.	Lucene  Dummies mbuy	0.192750
3.	Managing Megabytes lucene	0.192750
